# Biosynthesis of Psylocybin in <i>Aspergillus niger</i>

## 1. Introduction

### 1.1 Literature review of the compound (<500 words)

### 1.2 Literature review of the cell factory (<500 words)

## 2. Problem definition (<300 words)

## 3. *If Project category I:* Reconstruction of a new GSM for your cell factory host of interest (<1500 words)

or

## 3. *If Project category II:* Selection and assessment of existing GSM (<500 words)

## 4. Computer-Aided Cell Factory Engineering (<1500 words if Category II project; <500 words for Category I project)

In [7]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

model = read_sbml_model('../2018_ATCC1015_GSM.xml') #the model was found in (Brandl, J., Aguilar-Pontes, M.V., Schäpe, P. et al. - additional file 2) 

'' is not a valid SBML 'SId'.


In [8]:
model.medium

{'BOUNDARY_GLCe': 10.0,
 'BOUNDARY_H2Oe': 1000.0,
 'BOUNDARY_HNO3e': 1000.0,
 'BOUNDARY_O2e': 1000.0,
 'BOUNDARY_PIe': 1000.0,
 'BOUNDARY_SLFe': 1000.0,
 'BOUNDARY_Ke': 1000.0,
 'BOUNDARY_CAe': 1000.0}

#### Making the reactions

In [3]:
from cobra import Reaction, Metabolite

In [9]:
model.metabolites.TRP # tjekker lige at tryptophan befinder sig i cytosol

Metabolite identifier,TRP
Name,L-Tryptophan
Memory address,0x07f5b4084a880
Formula,C11H12N2O2
Compartment,c
In 8 reaction(s),"r1266, r1080, r400, r1265, PROTEINpolym, r385, r3008, r2402"


Jeg kan ikke finde tryptamin i modellen allerede (hverken som THR, THF, TRE, THY, TYR eller TPHTH og jeg synes ikke der er andre oplagte end dem jeg kan se - derfor tilføjer jeg sekv tryptamin som en metabolit:

Så derfor insættes metabolitterne også i compartment c (nedenfor)

Spørgsmål: burde vi også tilføje formula + name til metabolitterne?


In [10]:
tryptamine = Metabolite(id = 'TRYP', compartment='c')

In [11]:
hydroxytryptamine = Metabolite(id='_4HTRYP', compartment='c') 

In [12]:
norbaeocystin = Metabolite(id='NBC' , compartment='c') 

In [13]:
psilocybin = Metabolite(id='psi', compartment='c')

In [14]:
S_adenosyl_L_methionine = Metabolite(id='SAM', compartment='c')

In [15]:
S_adenosyl_L_homocysteine = Metabolite(id='SAH', compartment='c')

Først definerer jeg lige reaktionerne:

In [16]:
reaction_1 = Reaction('something_nice')  

In [17]:
reaction_2 = Reaction('something_nicer')

In [18]:
reaction_3 = Reaction('something_awesome')

In [19]:
reaction_4 = Reaction('Bjarne')

Nu tilføjer jeg alle metabolitterne til reaktionerne:

In [20]:
reaction_1.add_metabolites(({model.metabolites.TRP: -1,  
                              tryptamine: 1, 
                              model.metabolites.CO2: 1, 
                            }))

In [21]:
reaction_2.add_metabolites({tryptamine: -1,
                              model.metabolites.O2: -1,
                              model.metabolites.H: -2, 
                              hydroxytryptamine: 1, 
                              model.metabolites.H2O: 1
                             })

In [22]:
reaction_3.add_metabolites({hydroxytryptamine: -1,
                              model.metabolites.ATP: -1,
                              norbaeocystin: 1, 
                              model.metabolites.ADP: 1, # our newly created metabolites
                              model.metabolites.H: 1
                             })

In [23]:
reaction_4.add_metabolites(({norbaeocystin: -1,
                             S_adenosyl_L_methionine: -2, 
                             psilocybin: 1, 
                             S_adenosyl_L_homocysteine: 2,
                             model.metabolites.H: 2
                             }))

In [24]:
model.add_reactions([reaction_1])
model.add_reactions([reaction_2])
model.add_reactions([reaction_3])
model.add_reactions([reaction_4])

In [25]:
model.add_boundary(model.metabolites.TRYP, type='demand')
model.add_boundary(model.metabolites._4HTRYP, type='demand')
model.add_boundary(model.metabolites.NBC, type='demand')
model.add_boundary(model.metabolites.psi, type='demand')
model.add_boundary(model.metabolites.SAM, type='demand')
model.add_boundary(model.metabolites.SAH, type='demand')

Reaction identifier,DM_SAH
Name,S-Adenosyl-L-homocysteine demand
Memory address,0x07f5b58735130
Stoichiometry,SAH --> S-Adenosyl-L-homocysteine -->
GPR,
Lower bound,0
Upper bound,1000.0


In [26]:
with model:
    model.objective = model.reactions.something_nice
    max_1 = model.optimize().objective_value
    
max_1

3.870967741935476

In [27]:
with model:
    model.objective = model.reactions.something_nicer
    max_2 = model.optimize().objective_value
    
max_2

3.5294117647058663

In [28]:
with model:
    model.objective = model.reactions.something_awesome
    max_3 = model.optimize().objective_value
    
max_3

3.529411764705867

In [29]:
with model:
    model.objective = model.reactions.Bjarne
    max_4 = model.optimize().objective_value
    
max_4

3.1395808620008148

In [30]:
for gene in model.genes.query('trp', 'name'):
    print(gene.name)

trpC
trpB homolog


In [31]:
essentiality = {}
for gene in model.genes:
    with model:
        gene.knock_out()
        essentiality[gene] = model.slim_optimize(error_value=0.)

In [32]:
essentiality2 = {}
for gene in model.genes:
    with model:
        gene.knock_out()
        essentiality2[gene] = model.optimize()

In [33]:
import operator

In [34]:
max(essentiality, key=essentiality.get)

Gene identifier,1144613
Name,adk1
Memory address,0x07f5b40625af0
Functional,True
In 7 reaction(s),"r541, r436, r486, r491, r543, r444, r542"


In [37]:
def keywithmaxval(d):
     """ a) create a list of the dict's keys and values; 
         b) return the key with the max value"""  
     v=list(d.values())
     k=list(d.keys())
     return k[v.index(max(v))]

In [38]:
keywithmaxval(essentiality)

Gene identifier,1144613
Name,adk1
Memory address,0x07f5b40625af0
Functional,True
In 7 reaction(s),"r541, r436, r486, r491, r543, r444, r542"


In [39]:
all_values = essentiality.values()
max_value = max(all_values)



print(max_value)

0.9398547241513246


In [40]:
with model as pgk_mutant:
    pgk_mutant.genes.b2926.knock_out()
    print(pgk_mutant.slim_optimize())

AttributeError: DictList has no attribute or entry b2926

In [43]:
with model:
    model.objective = model.reactions.r385
    print(model.optimize().objective_value)

48.218623481780675


In [44]:
with model:
        model.objective = model.reactions.r385 
        for gene in model.genes:
            gene.knock_out()
            if model.optimize().objective_value > 48.2:
                print(gene)

1128503
1145456
1155770


In [45]:
for gene in model.genes:
    with model:
        gene.knock_out()
        model.objective = model.reactions.r385 
        if model.optimize().objective_value > 48:
            print(gene)

1128503
1145456
1155770
1143538
1126547
1128684
1146884
1145051
1141976
1144773
1145895
1124768
1183468
1031996
1120162
1147250
1145327
1142568
1139065
178461
1144775
1167742
1144228
1184284
1185167
1141647
1114699
1148603
1175567
1181034
1218960
1148375
1148025
1146854
1037649
1144385
1145655
1141712
1215111
1136340
1133804
1143651
1144118
48047
1172632
1102128
1183763
1103567
1147138
1111634
1117686
1145545
1164153
1137712
1145644
1151044
1118068
1082822
1106057
1121815
1153588
1187201
1189342
1180995
1187871
1183555
1082059
1149680
42017
1176225
1124003
1176774
1173314
1144646
1005336
1019169
1112134
1144124
1145307
1088723
1114868
1131969
1122220
1162909
1098004
1147280
52241
1156430
214187
1148469
1126514
1169517
1123538
1157856
1155201
1143782
1143776
1155794
1119507
1148545
1106635
1157820
1145607
1183753
1115259
1184557
1186265
1132937
1086134
1186361
1186757
1126169
1135615
1110758
46070
1121861
1099711
1160431
1180911
1168383
178804
1128706
1185624
1107471
1175582
1014459
110

In [46]:
for reaction in model.metabolites.TRP.reactions:
    print(reaction.id, reaction, reaction.name)

r1266 r1266: 0.951 ALA + 0.307 ARG + 0.366 ASN + 0.644 ASP + 40.39 ATP + 0.146 CYS + 0.249 GLN + 0.38 GLU + 0.687 GLY + 0.059 HIS + 0.351 ILE + 0.702 LEU + 0.19 LYS + 0.044 MET + 0.322 PHE + 0.322 PRO + 1.272 SER + 1.082 THR + 0.278 TRP + 0.395 TYR + 0.614 VAL --> 40.39 ADP + GAMYLe + 40.49 PI Artifical protein synthesis reaction
r1080 r1080: TRPe <=> TRP 
r400 r400: O2 + TRP --> FKYN Indoleamine 2,3-dioxygenases, Tryptophan 2,3-dioxygenase
r1265 r1265: 0.622 ALA + 0.226 ARG + 0.452 ASN + 0.773 ASP + 39.37 ATP + 0.17 CYS + 0.283 GLN + 0.32 GLU + 0.754 GLY + 0.132 HIS + 0.509 ILE + 0.716 LEU + 0.189 LYS + 0.151 MET + 0.264 PHE + 0.358 PRO + 1.018 SER + 0.716 THR + 0.207 TRP + 0.66 TYR + 0.603 VAL --> AAMYLe + 39.37 ADP + 39.37 PI Artifical protein synthesis reaction
PROTEINpolym PROTEINpolym: 1.033 ALA + 0.413 ARG + 0.209 ASN + 0.628 ASP + 39.729 ATP + 0.079 CYS + 0.3 GLN + 0.899 GLU + 0.862 GLY + 30.489 H2O + 0.205 HIS + 0.396 ILE + 0.704 LEU + 0.651 LYS + 0.105 MET + 0.312 PHE + 0.441

In [47]:
for reaction in model.reactions.query('total', 'name'):
    print(reaction.name)

In [48]:
model.reactions.DRAIN_Biomass

Reaction identifier,DRAIN_Biomass
Name,
Memory address,0x07f5b3a1a4a00
Stoichiometry,BIOMASS --> A. niger biomass -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [49]:
model.reactions.r385.flux


48.218623481780696

In [50]:
knockout_fluxes = {}
for gene in model.genes:
    with model:
        gene.knock_out()
        solution = model.reactions.r385
        if model.slim_optimize(error_value=0.) != 0:
            knockout_fluxes[gene] = solution.flux

knockout_fluxes

{<Gene 1128503 at 0x7f5b406b7f70>: 0.03114490584297179,
 <Gene 1145456 at 0x7f5b406b7040>: 0.03114490584297179,
 <Gene 1155770 at 0x7f5b406b70a0>: 0.03114490584297179,
 <Gene 1143538 at 0x7f5b406b7e80>: 0.03114490584297179,
 <Gene 1146031 at 0x7f5b40650040>: 0.030675459465383883,
 <Gene 1126547 at 0x7f5b40650100>: 0.031144905848857627,
 <Gene 1128684 at 0x7f5b406500a0>: 0.030148945655481717,
 <Gene 1146884 at 0x7f5b40650130>: 0.031144905848857755,
 <Gene 1145051 at 0x7f5b40650160>: 0.030148945655481717,
 <Gene 1141976 at 0x7f5b40650190>: 0.031144905848857755,
 <Gene 1144773 at 0x7f5b406501c0>: 0.031144905848857755,
 <Gene 1145793 at 0x7f5b406501f0>: 0.027732856004936073,
 <Gene 1148178 at 0x7f5b40650220>: 0.027732856004936073,
 <Gene 1145895 at 0x7f5b40650250>: 0.031144905848858075,
 <Gene 1124768 at 0x7f5b40650280>: 0.031144905848858075,
 <Gene 1183468 at 0x7f5b406502b0>: 0.031144905848858075,
 <Gene 1113563 at 0x7f5b406502e0>: 0.028397000115324263,
 <Gene 1184650 at 0x7f5b40650310>: 

In [51]:
knockout_fluxes1 = {}
for gene in model.genes:
    with model:
        gene.knock_out()
        solution = model.reactions.r385
        if solution.flux != 0:
            knockout_fluxes1[gene] = solution.flux

knockout_fluxes1

{<Gene 1128503 at 0x7f5b406b7f70>: 0.03114490584885552,
 <Gene 1145456 at 0x7f5b406b7040>: 0.03114490584885552,
 <Gene 1155770 at 0x7f5b406b70a0>: 0.03114490584885552,
 <Gene 1143538 at 0x7f5b406b7e80>: 0.03114490584885552,
 <Gene 1146031 at 0x7f5b40650040>: 0.03114490584885552,
 <Gene 1126547 at 0x7f5b40650100>: 0.03114490584885552,
 <Gene 1128684 at 0x7f5b406500a0>: 0.03114490584885552,
 <Gene 1146884 at 0x7f5b40650130>: 0.03114490584885552,
 <Gene 1145051 at 0x7f5b40650160>: 0.03114490584885552,
 <Gene 1141976 at 0x7f5b40650190>: 0.03114490584885552,
 <Gene 1144773 at 0x7f5b406501c0>: 0.03114490584885552,
 <Gene 1145793 at 0x7f5b406501f0>: 0.03114490584885552,
 <Gene 1148178 at 0x7f5b40650220>: 0.03114490584885552,
 <Gene 1145895 at 0x7f5b40650250>: 0.03114490584885552,
 <Gene 1124768 at 0x7f5b40650280>: 0.03114490584885552,
 <Gene 1183468 at 0x7f5b406502b0>: 0.03114490584885552,
 <Gene 1113563 at 0x7f5b406502e0>: 0.03114490584885552,
 <Gene 1184650 at 0x7f5b40650310>: 0.03114490584

In [52]:
keywithmaxval(knockout_fluxes1)

Gene identifier,1128503
Name,56084
Memory address,0x07f5b406b7f70
Functional,True
In 4 reaction(s),"r4, r2, r1, r3"


In [53]:
for gene in knockout_fluxes1:
    if gene == 1128503:
        print(gene)

In [54]:
model.reactions.r376.flux

0.1277929866972881

## 5. Discussion (<500 words)

## 6. Conclusion (<200 words)

## References
Brandl, J., Aguilar-Pontes, M.V., Schäpe, P. et al. A community-driven reconstruction of the Aspergillus niger metabolic network. Fungal Biol Biotechnol 5, 16 (2018). https://doi.org/10.1186/s40694-018-0060-7